In [80]:
import pandas as pd
import glob
import os
import random
import numpy as np
import cv2


In [81]:
df=pd.read_csv("../Data/data.csv")
df.head()

,Patient,RNASeqCluster,MethylationCluster,miRNACluster,CNCluster,RPPACluster,OncosignCluster,COCCluster,histological_type,neoplasm_histologic_grade,tumor_tissue_site,laterality,tumor_location,gender,age_at_initial_pathologic,race,ethnicity,death01
0,TCGA_CS_4941,2.0,4.0,2,2.0,NaN,3.0,2,1.0,2.0,1.0,3.0,2.0,2.0,67.0,3.0,2.0,1.0
1,TCGA_CS_4942,1.0,5.0,2,1.0,1.0,2.0,1,1.0,2.0,1.0,3.0,2.0,1.0,44.0,2.0,NaN,1.0
2,TCGA_CS_4943,1.0,5.0,2,1.0,2.0,2.0,1,1.0,2.0,1.0,1.0,2.0,2.0,37.0,3.0,NaN,0.0
3,TCGA_CS_4944,NaN,5.0,2,1.0,2.0,1.0,1,1.0,1.0,1.0,3.0,6.0,2.0,50.0,3.0,NaN,0.0
4,TCGA_CS_5393,4.0,5.0,2,1.0,2.0,3.0,1,1.0,2.0,1.0,1.0,6.0,2.0,39.0,3.0,NaN,0.0


In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Patient                    110 non-null    object 
 1   RNASeqCluster              92 non-null     float64
 2   MethylationCluster         109 non-null    float64
 3   miRNACluster               110 non-null    int64  
 4   CNCluster                  108 non-null    float64
 5   RPPACluster                98 non-null     float64
 6   OncosignCluster            105 non-null    float64
 7   COCCluster                 110 non-null    int64  
 8   histological_type          109 non-null    float64
 9   neoplasm_histologic_grade  109 non-null    float64
 10  tumor_tissue_site          109 non-null    float64
 11  laterality                 109 non-null    float64
 12  tumor_location             109 non-null    float64
 13  gender                     109 non-null    float64

In [83]:
data_map = []

for sub_dir_path in glob.glob("/workspaces/Projet_RGD/Data/kaggle_3m/*"):
    try:
        dir_name = os.path.basename(sub_dir_path)
        for filename in os.listdir(sub_dir_path):
            image_path = os.path.join(sub_dir_path, filename)
            data_map.append([dir_name, image_path])
    except Exception as e:
        print(e)

In [84]:
df1 = pd.DataFrame(data_map, columns=["patient_id", "image_path"])
df1.head()

,patient_id,image_path
0,TCGA_FG_5964_20010511,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_FG_...
1,TCGA_FG_5964_20010511,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_FG_...
2,TCGA_FG_5964_20010511,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_FG_...
3,TCGA_FG_5964_20010511,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_FG_...
4,TCGA_FG_5964_20010511,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_FG_...


In [85]:
df_imgs = df1[~df1['image_path'].str.contains("mask")]
df_masks = df1[df1['image_path'].str.contains("mask")]

# extraction dfu numéro de l'image 
BASE_LEN = 76 # index ou commence le numéro
END_IMG_LEN = 4 # nombre de chaines de caractères à éliminer de la fin du chemin de l'image
END_MASK_LEN = 9 # nombre de chaines de caractères à éliminer de la fin du chemin du mask

# trier les images et masks selon leur numéros
imgs = sorted(df_imgs["image_path"].values, key=lambda x : int(x[BASE_LEN:-END_IMG_LEN]))
masks = sorted(df_masks["image_path"].values, key=lambda x : int(x[BASE_LEN:-END_MASK_LEN]))

#vérification 
idx = random.randint(0, len(imgs)-1)
print("Path to the Image:", imgs[idx], "\nPath to the Mask:", masks[idx])

Path to the Image: /workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_A61B_19991127/TCGA_HT_A61B_19991127_76.tif 
Path to the Mask: /workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_A61B_19991127/TCGA_HT_A61B_19991127_76_mask.tif


# Creation de la data finale 

In [86]:
df_final = pd.DataFrame({
    "patient_id": df_imgs.patient_id.values,
    "image_path": imgs,
    "mask_path": masks
})

def pos_neg_diagnosis(mask_path):
    value = np.max(cv2.imread(mask_path))
    if value > 0 : 
        return 1
    else:
        return 0

In [87]:
df_final['mask'] = df_final['mask_path'].apply(lambda x: pos_neg_diagnosis(x))

In [88]:
df_final

,patient_id,image_path,mask_path,mask
0,TCGA_FG_5964_20010511,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_...,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_...,0
1,TCGA_FG_5964_20010511,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_...,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_...,0
2,TCGA_FG_5964_20010511,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_...,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_...,0
3,TCGA_FG_5964_20010511,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_...,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_...,0
4,TCGA_FG_5964_20010511,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_...,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_...,0
...,...,...,...,...
3924,TCGA_DU_5854_19951104,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_...,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_...,0
3925,TCGA_DU_5854_19951104,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_...,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_...,0
3926,TCGA_DU_5854_19951104,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_...,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_...,0
3927,TCGA_DU_5854_19951104,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_...,/workspaces/Projet_RGD/Data/kaggle_3m/TCGA_HT_...,0
